In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'e:\\project\\Customer Purchase Prediction'

In [3]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
import mlflow
from mlflow.tracking import MlflowClient
from src.logger import logging

In [4]:
import dagshub
dagshub.init(repo_owner='PhamAnhTienn', repo_name='Sleep-Efficiency-App', mlflow=True)

Accessing as PhamAnhTienn

Initialized MLflow to track repo "PhamAnhTienn/Sleep-Efficiency-App"

Repository PhamAnhTienn/Sleep-Efficiency-App initialized!

In [5]:
model_name = "sleep_efficiency_model"

class InferrencePineline:
    def __init__(self):
        self.model = joblib.load(Path('artifacts/model_trainer/model.joblib'))
    
    def load_latest_model(self):
        mlflow.set_registry_uri("https://dagshub.com/PhamAnhTienn/Sleep-Efficiency-App.mlflow")
        client = MlflowClient()
        
        logging.info(f"Attempting to load latest version of model: {model_name}")
        latest_version = max(
            client.search_model_versions(f"name='{model_name}'"),
            key=lambda mv: int(mv.version)
        )
        latest_version_number = latest_version.version
        run_id = latest_version.run_id
        logging.info(
            f"Found model version: {latest_version_number} with run_id: {run_id}"
        )
        
        if not run_id:
            logging.error("No run_id found for the model version")
            return None, None

        # Load the model
        model = mlflow.pyfunc.load_model(f"models:/{model_name}/{latest_version_number}")
        logging.info("Model loaded successfully")
        
        self.model = model
        joblib.dump(model, os.path.join("artifacts/model_trainer", "model.joblib"))
        logging.info("Model saved to artifacts/model_trainer")
        
    def predict(self, data):
        prediction = self.model.predict(data)
        
        return prediction

In [6]:
sample_data = {
    "Age": 25,
    "Gender": "Male",
    "Bedtime": 10.5,
    "Bedtime_AMPM": "PM",
    "Wakeup_time": 6.0,
    "Wakeup_time_AMPM": "AM",
    "Sleep_duration": 7.5,
    "Awakenings": 1,
    "Caffeine_consumption": "No",
    "Alcohol_consumption": "Yes",
    "Smoking_status": "No",
    "Exercise_frequency": 3
}

# Tiền xử lý dữ liệu như trong đoạn code Flask
Age = float(sample_data['Age'])
Gender = 1 if sample_data['Gender'] == 'Male' else 0
Bedtime = float(sample_data['Bedtime'])
Bedtime_AMPM = sample_data['Bedtime_AMPM']
if Bedtime_AMPM == 'PM' and Bedtime < 12:
    Bedtime += 12
elif Bedtime_AMPM == 'AM' and Bedtime == 12:
    Bedtime = 0
Wakeup_time = float(sample_data['Wakeup_time'])
Wakeup_time_AMPM = sample_data['Wakeup_time_AMPM']
if Wakeup_time_AMPM == 'PM' and Wakeup_time < 12:
    Wakeup_time += 12
elif Wakeup_time_AMPM == 'AM' and Wakeup_time == 12:
    Wakeup_time = 0
Sleep_duration = float(sample_data['Sleep_duration'])
Awakenings = float(sample_data['Awakenings'])
Caffeine_consumption = 1 if sample_data['Caffeine_consumption'] == 'Yes' else 0
Alcohol_consumption = 1 if sample_data['Alcohol_consumption'] == 'Yes' else 0
Smoking_status = 1 if sample_data['Smoking_status'] == 'Yes' else 0
Exercise_frequency = 1 if float(sample_data['Exercise_frequency']) > 1 else 0

input_data = [
    Age, Gender, Bedtime, Wakeup_time, Sleep_duration, 
    Awakenings, Caffeine_consumption, Alcohol_consumption,
    Smoking_status, Exercise_frequency
]

feature_names = [
    'Age',
    'Gender',
    'Bedtime',
    'Wakeup time',
    'Sleep duration',
    'Awakenings',
    'Caffeine consumption',
    'Alcohol consumption',
    'Smoking status',
    'Exercise frequency'
]

# Chuyển đổi sang DataFrame để đưa vào model
df = pd.DataFrame([input_data], columns=feature_names)

pipeline = InferrencePineline()
#pipeline.load_latest_model()
prediction = pipeline.predict(df)
print(f"Prediction result: {prediction[0]:.2f}")

Prediction result: 0.70
